In [1]:
import torch 
import torch.nn as nn
import os
import pandas as pd
import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [2]:
root_path = r'/kaggle/input/fer2013'
print(os.listdir(root_path))

['test', 'train']


In [3]:
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

class facial(Dataset):
    def __init__(self, root_path, transform = None):
        self.train_folder = []
        self.test_folder = []
        self.data = []
        self.transform = transform
        for folder in (os.listdir(root_path)):
            sub_folder = os.path.join(root_path, folder)
            if "test" in sub_folder:
                self.test_folder.append(sub_folder)
            else:
                self.train_folder.append(sub_folder)

        print("Train folder:  ", self.train_folder)
        
        classes = []

        for train_folder in self.train_folder:
            for class_folder in (os.listdir(train_folder)):
                # this will append the name of the subfolders in string format
                classes.append(class_folder)

        # string format in list class.
        classes = sorted(classes)
        print(classes)

        # Give each class a number.
        class_index = {}
        for i in range(len(classes)):
            class_index[classes[i]] = i
        print(class_index)
       
        # now we want to insert the images to that particular class.
        for train_folder in self.train_folder:
            # sort the folder before inserting into the class
            for class_folder in sorted(os.listdir(train_folder)):
                image_folder = os.path.join(train_folder, class_folder)
                labels = class_index[class_folder]
                for img in (os.listdir(image_folder)):
                    img_path = os.path.join(image_folder, img)
                    self.data.append((img_path, labels))
     
    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        # here self.data is already a tuple
        img_path, labels = self.data[index]
        # convert the image to RGB
        img = Image.open(img_path).convert("L")
        
        if self.transform:
            img = self.transform(img)

        return img, labels

dataset = facial(root_path)

Train folder:   ['/kaggle/input/fer2013/train']
['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
{'angry': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 'neutral': 4, 'sad': 5, 'surprise': 6}


In [1]:
# Now we got the dataset and its ready for loading.

all_images = []
all_labels = []
for images, labels in dataset.data:
    all_images.append(images)
    all_labels.append(labels)
print("All Images : ", len(all_images))
print("All labels : ", set(all_labels))

In [5]:
# we got the train and test image with loaders 
# from this we will get separate train image paths and labels.

from PIL import Image
class Train(Dataset):
    def __init__(self, path, labels, transform = None):
        self.image_path = path
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_path)

    def __getitem__(self, index):
        image = self.image_path[index]
        labels = self.labels[index]
        # grayscale image
        img = Image.open(image).convert("L")
        if self.transform:
            img = self.transform(img)
        return img, labels



In [6]:
from collections import Counter

transform = transforms.Compose([transforms.Resize((48, 48)),transforms.ToTensor(),
                               transforms.Normalize((0.5,), (0.5,))])

train_image, test_image, train_labels, test_labels = train_test_split(all_images, all_labels, test_size = 0.2)

train_dataset = Train(train_image, train_labels, transform = transform)
test_dataset = Train(test_image, test_labels, transform = transform)

In [7]:
label_counts = Counter(train_labels)

# Check how many samples are allocated to the labels.
print("Train labels distribution:", Counter(train_labels))

class_sample_count = [label_counts[i] for i in range(len(label_counts))]
print("Class sample count : ", class_sample_count)

Train labels distribution: Counter({3: 5828, 4: 3936, 5: 3866, 2: 3301, 0: 3160, 6: 2543, 1: 333})
Class sample count :  [3160, 333, 3301, 5828, 3936, 3866, 2543]


In [8]:
# ---------- WeightedRandomSampler to fix class imbalance ---------------------
# Label 1 (disgust) is having small examples - this denotes this dataset has class imbalance.

# we are dividing it => 1 / 357 (class_count)so the class 1 has higher weight
weights = 1. / torch.tensor(class_sample_count, dtype=torch.float)
print("Weight : ", weights)

samples_weight = [weights[label] for label in train_labels]
print("Samples weight : ", samples_weight[:7])

sampler = WeightedRandomSampler(samples_weight, num_samples=len(samples_weight), replacement=True)

Weight :  tensor([0.0003, 0.0030, 0.0003, 0.0002, 0.0003, 0.0003, 0.0004])
Samples weight :  [tensor(0.0003), tensor(0.0004), tensor(0.0003), tensor(0.0003), tensor(0.0003), tensor(0.0003), tensor(0.0003)]


In [9]:
# Not using shuffle here because class 1 samples has to picked with higher probability.
train_dataloader = DataLoader(dataset = train_dataset, sampler = sampler, batch_size = 30)
test_loader = DataLoader(dataset = test_dataset, shuffle = False, batch_size = 30)

print("Total batch size of train loader: ", len(train_dataloader))
print("Total length of test loader : ", len(test_loader))

examples = iter(train_dataloader)
samples, labels = next(examples)
print(f"Sample shape : {samples.shape}, labels shape :  {labels.unique()}")

Total batch size of train loader:  766
Total length of test loader :  192
Sample shape : torch.Size([30, 1, 48, 48]), labels shape :  tensor([0, 1, 2, 3, 4, 5, 6])


In [10]:
# Define the neural network

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 5)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv2d(32, 64, 5)
        self.pool = nn.MaxPool2d(2,2)
        self.fc1 = nn.Linear(64*9*9, 120)
        self.fc2 = nn.Linear(120, 64)
        self.fc3 = nn.Linear(64, 7)

    def forward(self, x):
        out_conv1 = self.relu(self.conv1(x))
        out_pool = self.relu(self.pool(out_conv1))
        out_conv2 = self.relu(self.conv2(out_pool))
        out_pool2 = self.relu(self.pool(out_conv2))

        # flatten the image
        output = out_pool2.view(-1, 64*9*9)

        out_fc1 = self.relu(self.fc1(output))
        out_fc2 = self.relu(self.fc2(out_fc1))
        out_fc3 = self.fc3(out_fc2)

        return out_fc3

model = NeuralNetwork()

learning_rate = 0.001

# loss and optimizer
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [14]:
total_steps = len(train_dataloader)
total_epoch = 5
epoch_loss = 0
for epoch in range(total_epoch):
    for i, (images, labels) in enumerate (train_dataloader):
        output = model(images)
        criterion = loss(output, labels)

        optimizer.zero_grad()
        criterion.backward()
        optimizer.step()

        batch_loss = criterion.item()
        
        epoch_loss += batch_loss

        if (i + 1) % 200 == 0:
            print(f"Epoch {epoch+1}, Step {i+1}/{total_steps}, Batch Loss = {batch_loss:.5f}")


Epoch 1, Step 200/766, Batch Loss = 0.15263
Epoch 1, Step 400/766, Batch Loss = 0.23042
Epoch 1, Step 600/766, Batch Loss = 0.17309
Epoch 2, Step 200/766, Batch Loss = 0.13871
Epoch 2, Step 400/766, Batch Loss = 0.12783
Epoch 2, Step 600/766, Batch Loss = 0.30829
Epoch 3, Step 200/766, Batch Loss = 0.16196
Epoch 3, Step 400/766, Batch Loss = 0.09367
Epoch 3, Step 600/766, Batch Loss = 0.24208
Epoch 4, Step 200/766, Batch Loss = 0.14752
Epoch 4, Step 400/766, Batch Loss = 0.20722
Epoch 4, Step 600/766, Batch Loss = 0.33512
Epoch 5, Step 200/766, Batch Loss = 0.29840
Epoch 5, Step 400/766, Batch Loss = 0.15262
Epoch 5, Step 600/766, Batch Loss = 0.13661


In [ ]:
# loss is fluctuating, so we will find the test accuracy

with torch.no_grad():
    correct = 0
    total = 0
    for image, labels in test_loader :
        output = model(image)
        _, predictions = torch.max(output, 1)
        correct += (predictions == labels).sum().item()
        total += labels.size(0)

    val_accuracy = 100 * correct / total
    print(f"Validation accuracy after epoch : {val_accuracy}")
        